#### Initialization

In [1]:
import findspark
findspark.init( '/usr/local/spark' )
import pyspark
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession, DataFrame, functions as f
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier, MultilayerPerceptronClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.types import FloatType

In [2]:
sc = SparkSession.builder.master( 'local' ).appName( 'Fraud Detection' ).getOrCreate()
print( sc.version )

2.4.0


#### Reading the Dataset

In [8]:
d = sc.read.csv( '/home/demetrius/Documents/Python/Fraud Detection/Data/fraud_dyninno.csv',
                inferSchema=True, header=True, quote='' )

In [9]:
d = d.withColumnRenamed( 'fraud', 'label' )
d = d.withColumnRenamed( 'amount_oringinal', 'amount' )
d = d.withColumnRenamed( 'term_oringinal', 'term' )
d = d.withColumnRenamed( 'r_application_id', 'id' )
d = d.withColumnRenamed( 'created', 'date' )
d = d.withColumnRenamed( 'income_per_month', 'income' )
d = d.withColumn( 'amount', d.amount / 91 * 2.2 )
d = d.withColumn( 'amount', f.round( 'amount', -1 ) )

In [10]:
p = sc.createDataFrame([( 'адыгея', 'Adygeya Respublika' ),
                        ( 'алтай', 'Altay Respublika' ), 
                        ( 'алтайский', 'Altayskiy kray' ), 
                        ( 'амурская', 'Amurskaya oblast' ), 
                        ( 'архангельская', 'Arkhangelskaya oblast' ), 
                        ( 'астраханская', 'Astrakhanskaya oblast' ), 
                        ( 'башкортостан', 'Bashkortostan Respublika' ), 
                        ( 'белгородская', 'Belgorodskaya oblast' ), 
                        ( 'брянская', 'Bryanskaya oblast' ), 
                        ( 'бурятия', 'Buryatiya Respublika' ), 
                        ( 'владимирская', 'Vladimirskaya oblast' ), 
                        ( 'волгоградская', 'Volgogradskaya oblast' ), 
                        ( 'вологодская', 'Vologodskaya oblast' ), 
                        ( 'воронежская', 'Voronezhskaya oblast' ), 
                        ( 'дагестан', 'Dagestan Respublika' ), 
                        ( 'дальневосточный', 'Saha Respublika' ), 
                        ( 'еврейская', 'Yevreyskaya avtonomnaya oblast' ), 
                        ( 'забайкальский', 'Zabaykalskiy kray' ), 
                        ( 'ивановская', 'Ivanovskaya oblast' ), 
                        ( 'ингушетия', 'Ingushetiya Respublika' ), 
                        ( 'иркутская', 'Irkutskaya oblast' ), 
                        ( 'кабардино-балкарская', 'Kabardino-Balkarskaya Respublika' ), 
                        ( 'калининградская', 'Kaliningradskaya oblast' ), 
                        ( 'калмыкия', 'Kalmykiya Respublika' ), 
                        ( 'калужская', 'Kaluzhskaya oblast' ), 
                        ( 'камчатский', 'Kamchatskiy kray' ), 
                        ( 'карачаево-черкесская', 'Karachayevo-Cherkesskaya Respublika' ), 
                        ( 'карелия', 'Kareliya Respublika' ), 
                        ( 'кемеровская', 'Kemerovskaya oblast' ), 
                        ( 'кемеровская-кузбасс', 'Kemerovskaya oblast' ), 
                        ( 'кировская', 'Kirovskaya oblast' ), 
                        ( 'коми', 'Komi Respublika' ), 
                        ( 'костромская', 'Kostromskaya oblast' ), 
                        ( 'краснодарский', 'Krasnodarskiy kray' ), 
                        ( 'красноярский', 'Krasnoyarskiy kray' ), 
                        ( 'курганская', 'Kurganskaya oblast' ), 
                        ( 'курская', 'Kurskaya oblast' ), 
                        ( 'ленинградская', 'Leningradskaya oblast' ), 
                        ( 'липецкая', 'Lipetskaya oblast' ), 
                        ( 'магаданская', 'Magadanskaya oblast' ), 
                        ( 'марий эл', 'Mariy El Respublika' ), 
                        ( 'мордовия', 'Mordoviya Respublika' ), 
                        ( 'москва', 'Moskva' ), 
                        ( 'московская', 'Moskovskaya oblast' ), 
                        ( 'мурманская', 'Murmanskaya oblast' ), 
                        ( 'ненецкий', 'Nenetskiy avtonomnyy okrug' ), 
                        ( 'нижегородская', 'Nizhegorodskaya oblast' ), 
                        ( 'новгородская', 'Novgorodskaya oblast' ), 
                        ( 'новосибирская', 'Novosibirskaya oblast' ), 
                        ( 'омская', 'Omskaya oblast' ), 
                        ( 'оренбургская', 'Orenburgskaya oblast' ), 
                        ( 'орловская', 'Orlovskaya oblast' ), 
                        ( 'пензенская', 'Penzenskaya oblast' ), 
                        ( 'пермский', 'Permskiy kray' ), 
                        ( 'приморский', 'Primorskiy kray' ), 
                        ( 'псковская', 'Pskovskaya oblast' ), 
                        ( 'ростовская', 'Rostovskaya oblast' ), 
                        ( 'рязанская', 'Ryazanskaya oblast' ), 
                        ( 'самарская', 'Samarskaya oblast' ), 
                        ( 'санкт-петербург', 'Sankt-Peterburg' ), 
                        ( 'саратовская', 'Saratovskaya oblast' ), 
                        ( 'саха /якутия/', 'Saha Respublika' ), 
                        ( 'сахалинская', 'Sakhalinskaya oblast' ), 
                        ( 'свердловская', 'Sverdlovskaya oblast' ), 
                        ( 'смоленская', 'Smolenskaya oblast' ), 
                        ( 'ставропольский', 'Stavropolskiy kray' ), 
                        ( 'тамбовская', 'Tambovskaya oblast' ), 
                        ( 'татарстан', 'Tatarstan Respublika' ), 
                        ( 'тверская', 'Tverskaya oblast' ), 
                        ( 'томская', 'Tomskaya oblast' ), 
                        ( 'тульская', 'Tulskaya oblast' ), 
                        ( 'тыва', 'Tyva Respublika' ), 
                        ( 'тюменская', 'Tyumenskaya oblast' ), 
                        ( 'удмуртская', 'Udmurtskaya Respublika' ), 
                        ( 'ульяновская', 'Ulyanovskaya oblast' ), 
                        ( 'хабаровский', 'Khabarovskiy kray' ), 
                        ( 'ханты-мансийский', 'Khanty-Mansiyskiy avtonomnyy okrug' ), 
                        ( 'челябинская', 'Chelyabinskaya oblast' ), 
                        ( 'чеченская', 'Chechenskaya Respublika' ), 
                        ( 'чувашская', 'Chuvashskaya Respublika' ), 
                        ( 'чувашская чувашия', 'Chuvashskaya Respublika' ), 
                        ( 'чукотский', 'Chukotskiy avtonomnyy okrug' ), 
                        ( 'ямало-ненецкий', 'Yamalo-Nenetskiy avtonomnyy okrug' ), 
                        ( 'ярославская', 'Yaroslavskaya oblast' ) ],
                         [ 'region_cleared', 'region_app' ] )
d = d.join( p, on='region_cleared', how='left' )
d = d.withColumn( 'calculator', f.when( d.cl_relationships_id == 2, 'right' ).\
    when( d.cl_relationships_id == 3, 'exact' ).when( d.cl_relationships_id.isNull(), 'default' ).otherwise( 'change' ) )
d = d.drop( 'region_cleared', 'mobile', 'desktop', 'windows', 'apple', 'linux', 'google', 
            'cl_relationships_id', 'income_null' )

#### Overview of the Dataset

In [11]:
nrows = d.count()
print( nrows )

17404


In [12]:
# len( d.columns )

In [13]:
#d.show( 1, vertical=True )

Let us count missing values by columns:

In [14]:
#d.select( [ f.sum( f.when( f.col( name ).isNull(), 1 ) ).alias( name ) for name in d.columns \
#           if d.select( name ).dtypes[ 0 ][ 1 ] in [ 'double', 'float', 'int' ] ] ).show( vertical=True )

#### Overview of the Target Variable (Label)

In [15]:
#d.select( 'label' ).distinct().show()

So the target variable is binary.

In [16]:
d.select( 'label' ).summary( 'mean' ).show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|   mean|0.04642610894047346|
+-------+-------------------+



The class is positive (actual fraud) in 4.6% of the cases. The distribution is rather imbalanced.

#### Financial Variables

"amount": amount of a loan (equivalent in euros)

In [17]:
#d.describe( 'amount' ).show()

"term" : term of a loan (in days)

In [18]:
#d.describe( 'term' ).show()

expected interest (given that the rate is 1% per day):

In [19]:
d = d.withColumn( 'interest', d.amount * 0.01 * d.term )

Additional versions of "amount" and "term" are created, which will be preprocessed:

In [20]:
d = d.withColumn( 'amount_prep', d.amount )
d = d.withColumn( 'term_prep', d.term )

#### Transformations of Some Variables

In [21]:
for name in [ 'cl_marital_status_id', 'cl_employment_type_id', 'cl_employment_area_id' ] :
    d = d.withColumn( name, f.col( name ).cast( 'string' ) )

In [22]:
#d.groupBy( 'ip_country_code' ).count().show()

In [23]:
d = d.withColumn( 'ip_country_code', f.when( d.ip_country_code == 'RU', 0 ).otherwise( 1 ) )

Provinces Mismatch:

In [24]:
d = d.withColumn( 'province_mismatch', 
 f.when( ( d.region_app != d.ip_region_name ) & d.region_app.isNotNull() & d.ip_region_name.isNotNull(), 
        1 ).otherwise( 0 ) )

Zip code:

In [25]:
d = d.withColumn( 'zip', f.substring( d.zip, 0, 3 ) )

In [26]:
d = d.withColumn( 'ip_zip_code', f.substring( d.ip_zip_code, 0, 3 ) )

In [27]:
d = d.withColumn( 'zip_mismatch', 
        f.when( ( d.zip != d.ip_zip_code ) & d.zip.isNotNull() & d.ip_zip_code.isNotNull(), 1 ).otherwise( 0 ) )

In [28]:
d.createOrReplaceTempView( 'd' )

In [29]:
q = "\
SELECT ip_mobile_brand, COUNT( * ), AVG( label ) \
FROM d \
GROUP BY 1 \
ORDER BY 2 DESC "
#sc.sql( q ).show()

Rare "ip_mobile_brand" are omitted in order not to introduce extra categories:

In [30]:
d = d.withColumn( 'ip_mobile_brand', f.when( d.ip_mobile_brand.isin([ 'MTS', 'MegaFon', 'Tele2', 'Win Mobile', 
                        '-', 'Beeline', 'MOTIV', 'TTK' ]), d.ip_mobile_brand ) )

In [31]:
q = "\
SELECT ip_usage_type, COUNT( * ), AVG( label ) \
FROM d \
GROUP BY 1 \
ORDER BY 2 DESC "
#sc.sql( q ).show()

Rare "ip_usage_type" are omitted in order not to introduce extra categories:

In [32]:
d = d.withColumn( 'ip_usage_type', f.when( d.ip_usage_type.isin([ 'DCH', 'MOB', 'ISP/MOB', 'ISP', 'COM' ]),
                                           d.ip_usage_type ) )

In [33]:
q = "\
SELECT ip_net_speed, COUNT( * ), AVG( label ) \
FROM d \
GROUP BY 1 \
ORDER BY 2 DESC "
#sc.sql( q ).show()

"-" in "ip_net_speed" is omitted in order not to introduce an extra category:

In [34]:
d = d.withColumn( 'ip_net_speed', f.when( d.ip_net_speed.isin([ 'T1', 'DSL', 'COMP', 'DIAL' ]), d.ip_net_speed ) )

"device_type":

In [35]:
q = "\
SELECT device_type, COUNT( * ), AVG( label ) \
FROM d \
GROUP BY 1 \
ORDER BY 2 DESC "
#sc.sql( q ).show()

A special variable for tablets:

In [36]:
d = d.withColumn( 'tablet', f.when( d.device_type == 'Tablet', 1 ).otherwise( 0 ) )

Timezone:

In [37]:
d = d.withColumn( 'local_time', f.from_utc_timestamp( d.date, d.ip_timezone_name ) )

In [38]:
d = d.withColumn( 'local_time', f.hour( d.local_time ) + f.minute( d.local_time ) / 60 )

In [39]:
d = d.withColumn( 'pixel_mismatch', 
      f.when( ( d.ip_timezone != d.pixel_timezone ) & d.ip_timezone.isNotNull() & d.pixel_timezone.isNotNull(), 
        1 ).otherwise( 0 ) )

Time is encoded as two variables with trigonometric functions:

In [40]:
d = d.withColumn( 'time_radian', d.local_time / 3.819718634205488 )

In [41]:
d = d.withColumn( 'time_sine', f.sin( d.time_radian ) )

In [42]:
d = d.withColumn( 'time_cosine', f.cos( d.time_radian ) )

Language:

In [43]:
q = "\
SELECT device_language, COUNT( * ), AVG( label ) \
FROM d \
GROUP BY 1 \
ORDER BY 2 DESC "
#sc.sql( q ).show()

In [44]:
d = d.withColumn( 'ru_lang', f.when( d.device_language == 'ru-ru', 1 ).otherwise( 0 ) )

In [45]:
d = d.withColumn( 'other_lang', f.when( d.device_language != 'ru-ru', 1 ).otherwise( 0 ) )

"pixel_language":

In [46]:
q = "\
SELECT pixel_language, COUNT( * ), AVG( label ) \
FROM d \
GROUP BY 1 \
ORDER BY 2 DESC "
#sc.sql( q ).show()

Rare "pixel_language" are omitted in order not to produce extra categories:

In [47]:
d = d.withColumn( 'pixel_language', f.when( d.pixel_language.isin([ 'ru-RU', 'ru', 'en-US', 'ru_ru', 'ru-Ru' ]), 
                                            d.pixel_language ) )

"device_year_release" is replaced by age of the device:

In [48]:
d = d.withColumn( 'device_age', f.year( d.date ) - d.device_year_release )

"os_name":

In [49]:
q = "\
SELECT os_name, COUNT( * ), AVG( label ) \
FROM d \
GROUP BY 1 \
ORDER BY 2 DESC "
#sc.sql( q ).show()

Rare "os_name" are omitted in order not to produce extra categories:

In [50]:
d = d.withColumn( 'os_name', f.when( d.os_name.isin([ 'Android', 'iOS', 'Windows 10', 'Windows 7', 'Linux', 
                                                      'Windows 8.1', 'OS X', 'Windows XP', 'Windows 8' ]), d.os_name ) )

"browser_name":

In [51]:
q = "\
SELECT browser_name, COUNT( * ), AVG( label ) \
FROM d \
GROUP BY 1 \
ORDER BY 2 DESC "
#sc.sql( q ).show()

Rare "browser_name" are omitted in order not to produce extra categories:

In [52]:
d = d.withColumn( 'browser_name', f.when( d.browser_name.isin([ 'Chrome Mobile', 'Safari', 'Yandex.Browser',
    'Chrome', 'Samsung Browser', 'MIUI Browser', 'Opera', 'Firefox', 
    'Opera Mobile', 'UC Browser', 'Edge' ]), d.browser_name ) )

"rendering_engine":

In [53]:
q = "\
SELECT rendering_engine, COUNT( * ), AVG( label ) \
FROM d \
GROUP BY 1 \
ORDER BY 2 DESC "
#sc.sql( q ).show()

"Trident" in "rendering_engine" is omitted in order not to introduce an extra category:

In [54]:
d = d.withColumn( 'rendering_engine', f.when( d.rendering_engine.isin([ 'WebKit', 'Blink', 'Gecko', 'UCWeb', 
                                                                        'EdgeHTML' ]), d.rendering_engine ) )

Variables with no or duplicate information are removed:

In [55]:
d = d.drop( 'additional_info', 'device_id', 'exact_device_id', 'browser_hash', 'user_id', 
  'idx7_device_applications_quality', 'shopping_vector_length', 'microlending_vector_length', 'gambling_vector_length', 
  'proxy_vector_length', 'photos', 'hdd_utils', 'ip_country_name', 'real_ip', 'real_ipv6', 'ip_first_seen_date',
  'dns_ip', 'ip_zip_code_distance', 'bot_name', 'device_model', 'device_marketing_name', 'device_manufacturer',
  'device_type', 'ip_timezone', 'ip_timezone_name', 'pixel_timezone', 'pixel_timezone_name', 'local_time', 'time_radian',
  'device_language', 'os_version', 'os_proprietary', 'development_platform', 'browser_version', 'device_year_release' )

#### Treatment of Extreme Values

In [56]:
#for name in d.columns :
#    d.select( name ).agg( f.countDistinct( f.col( name ) ), f.min( f.col( name ) ), f.max( f.col( name ) ) ).show()

In [57]:
numeric = [ 'amount_prep', 'term_prep', 'age', 'income', 'cl_education_type_id', 'antifraud_score', 'time', 'idx1_stop_markers', 'idx2_user_behaviour_markers', 'idx3_device_markers', 'idx4_connection_markers', 'idx5_device_quality', 'idx6_internet_infrastructure_quality', 'idx8_device_credentials_variability', 'total_applications_number', 'total_num_of_applications_with_browser_hash_in_1_day', 'total_num_of_applications_with_browser_hash_in_7_days', 'total_num_of_applications_with_browser_hash_in_30_days', 'logins_number', 'phones_number', 'devices_number', 'cities_number', 'zipcodes_number', 'cards_number', 'device_ageing_with_user', 'total_num_of_shortterm_credit_applications_in_1_day', 'total_num_of_shortterm_credit_applications_in_7_days', 'total_num_of_shortterm_credit_applications_in_30_days', 'total_num_of_shortterm_credit_applications_from_ip_in_1_day', 'total_num_of_shortterm_credit_applications_from_ip_in_7_days', 'total_num_of_shortterm_credit_applications_from_ip_in_30_days', 'total_num_of_banking_credit_applications_in_1_day', 'total_num_of_banking_credit_applications_in_7_days', 'total_num_of_banking_credit_applications_in_30_days', 'total_num_of_banking_credit_applications_from_ip_in_1_day', 'total_num_of_banking_credit_applications_from_ip_in_7_days', 'total_num_of_banking_credit_applications_from_ip_in_30_days', 'total_num_of_insurance_applications_from_ip_in_30_days', 'mail_login_length', 'time_on_page', 'number_of_hot_keys', 'number_of_corrections', 'browser_history_count', 'ram_productivity', 'storage_productivity', 'pure_avg_typing_speed', 'avg_typing_speed', 'single_click', 'double_click', 'context_menu', 'leave_page', 'observed_quarters', 'touch_dev', 'acc_dev', 'number_of_secondary_antifraud_markers', 'ip_longitude', 'ip_latitude', 'ip_ageing_in_months', 'is_canvas_blocker', 'display_width_via_pixel', 'display_height_via_pixel', 'display_width_from_manufacturer', 'display_height_from_manufacturer', 'usable_display_width', 'usable_display_height', 'diagonal_screen_size', 'display_ppi', 'display_pixel_ratio', 'color_depth_via_pixel', 'color_depth_from_manufacturer', 'ram_size', 'hardware_concurrency', 'camera_resolution', 'completeness', 'time_sine', 'time_cosine', 'device_age' ]

Extreme values of some numeric variables are winsorized (limited at a particular threshold). This is done with the following algorithm:

In [58]:
for name in numeric :
    # Calculates the main percentiles
    q1, q2, q3 = d.approxQuantile( name, [ 0.25, 0.5, 0.75 ], 0.001 )
    # Checks if the distribution is not very dense
    if ( q2 - q1 ) != 0 and ( q3 - q2 ) != 0 :
        # Checks if the distribution is non-symmetrical
        skewed = ( q3 - q2 ) / ( q2 - q1 ) >= 1.25 or ( q2 - q1 ) / ( q3 - q2 ) >= 1.25
    else:
        skewed = True
    # Threshold values for a skewed distribution
    if skewed and d.agg( f.min( f.col( name ) ) ).collect()[ 0 ][ 0 ] >= 0 : 
        lower = 0
        if q3 > 0 :
            upper = q3 * 2   
        else :
            upper = 1
    # Threshold values for a symmetrical distribution
    else :
        iqr = q3 - q1
        if iqr > 0 :
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr
        else :
            lower = q1 - 1
            upper = q3 + 1
    # Prints the threshold values
    print( ( name, lower, upper ) )
    # Replaces extreme values with the threshold values
    d = d.withColumn( name, f.when( f.col( name ) < lower, lower ).when( f.col( name ) > upper, upper ).\
                            otherwise( f.col( name ) ) )

('amount_prep', -240.0, 720.0)
('term_prep', 0, 60.0)
('age', 0, 73.14989733059548)
('income', 0, 108000.0)
('cl_education_type_id', 0, 24.0)
('antifraud_score', 0, 0.7015)
('time', 0, 5.094266)
('idx1_stop_markers', 0, 1)
('idx2_user_behaviour_markers', 0, 10.0)
('idx3_device_markers', 0, 1)
('idx4_connection_markers', 0, 2.0)
('idx5_device_quality', -2.0, 6.0)
('idx6_internet_infrastructure_quality', 0, 2.0)
('idx8_device_credentials_variability', 0, 6.0)
('total_applications_number', -3.0, 5.0)
('total_num_of_applications_with_browser_hash_in_1_day', 0, 2.0)
('total_num_of_applications_with_browser_hash_in_7_days', 0, 16.0)
('total_num_of_applications_with_browser_hash_in_30_days', 0, 46.0)
('logins_number', 0, 2.0)
('phones_number', 0, 1)
('devices_number', 0, 1)
('cities_number', 0, 2.0)
('zipcodes_number', 0, 2.0)
('cards_number', 0, 1)
('device_ageing_with_user', 0, 2.0)
('total_num_of_shortterm_credit_applications_in_1_day', 0, 1)
('total_num_of_shortterm_credit_applications_in

#### Train / Test Split

In [59]:
border_positive = d.filter( 'label == 1' ).select( 'id' ).approxQuantile( 'id', [ 0.8 ], 0.001 )[ 0 ]

In [60]:
border_negative = d.filter( 'label == 0' ).select( 'id' ).approxQuantile( 'id', [ 0.8 ], 0.001 )[ 0 ]

In [61]:
d = d.withColumn( 'test_positive', f.when( d.id > border_positive, True ).otherwise( False ) )

In [62]:
d = d.withColumn( 'test_negative', f.when( d.id > border_negative, True ).otherwise( False ) )

In [63]:
d = d.withColumn( 'test', f.when( d.label == 1, d.test_positive ).otherwise( d.test_negative ) )

In [64]:
#d.groupBy( 'test' ).agg( f.count( 'label' ), f.mean( 'label' ) ).show()

In [65]:
d = d.drop( 'test_positive', 'test_negative' )

#### Scaling

Some of the variables degenerated into binary after the treatment of extreme values, so the list should be updated.

In [66]:
degenerated = [ 'total_num_of_shortterm_credit_applications_from_ip_in_1_day', 'total_num_of_banking_credit_applications_in_1_day', 'total_num_of_banking_credit_applications_in_7_days', 'total_num_of_banking_credit_applications_in_30_days', 'total_num_of_banking_credit_applications_from_ip_in_1_day', 'total_num_of_banking_credit_applications_from_ip_in_7_days', 'total_num_of_insurance_applications_from_ip_in_30_days', 'mail_login_length', 'number_of_hot_keys', 'number_of_corrections', 'total_num_of_shortterm_credit_applications_in_7_days', 'total_num_of_shortterm_credit_applications_in_1_day', 'cards_number', 'devices_number', 'phones_number', 'idx3_device_markers', 'idx1_stop_markers' ]

In [67]:
numeric = [ x for x in numeric if x not in degenerated ]

All numeric variables are scaled to the interval [0, 1] with sigmpoid function and standardizing. Estimation of means and standard deviations should be done only on the train sample to prevent overfit. 

In [68]:
for name in numeric :
    m = d.filter( 'test = false' ).select( f.mean( f.col( name ) ) ).collect()[ 0 ][ 0 ]
    sd = d.filter( 'test = false' ).select( f.stddev( f.col( name ) ) ).collect()[ 0 ][ 0 ]
    d = d.withColumn( name, ( f.col( name ) - m ) / sd )
    d = d.withColumn( name, 1 / ( 1 + f.exp( -f.col( name ) ) ) )

#### Imputation of Missing Values

Missing values in categorical and binary variables become a new category ("Unknown"):

In [69]:
categorical = [ 'total_num_of_shortterm_credit_applications_from_ip_in_1_day', 'total_num_of_banking_credit_applications_in_1_day', 'total_num_of_banking_credit_applications_in_7_days', 'total_num_of_banking_credit_applications_in_30_days', 'total_num_of_banking_credit_applications_from_ip_in_1_day', 'total_num_of_banking_credit_applications_from_ip_in_7_days', 'total_num_of_insurance_applications_from_ip_in_30_days', 'mail_login_length', 'number_of_hot_keys', 'number_of_corrections', 'total_num_of_shortterm_credit_applications_in_7_days', 'total_num_of_shortterm_credit_applications_in_1_day', 'cards_number', 'devices_number', 'phones_number', 'idx3_device_markers', 'idx1_stop_markers', 'gender', 'cl_marital_status_id', 'cl_employment_type_id', 'cl_employment_area_id', 'zip', 'javascript_executable', 'ip_available', 'useragent_available', 'supporting_data_available', 'is_local_country', 'timezone_mismatch', 'ip_sim_regions_match', 'duplicating_device', 'duplicating_ip', 'duplicating_user', 'seen_before', 'same_phone', 'same_login', 'same_region', 'same_device', 'less_tenor_days', 'total_num_of_insurance_applications_in_1_day', 'total_num_of_insurance_applications_in_7_days', 'total_num_of_insurance_applications_in_30_days', 'total_num_of_insurance_applications_from_ip_in_1_day', 'total_num_of_insurance_applications_from_ip_in_7_days', 'foreign_language_used', 'frd', 'frd_from_diff_acc', 'npl90', 'npl90_from_diff_acc', 'is_ever_frd', 'is_mass_frd', 'global_ip_blacklist', 'global_isp_blacklist', 'user_defined_ip_blacklist', 'user_defined_deviceid_blacklist', 'user_defined_userid_blacklist', 'user_defined_isp_blacklist', 'user_defined_similar_deviceid_blacklist', 'suspicious_referral', 'suspicious_plugins', 'suspicious_fonts', 'cookie_clone', 'mail_login_marker', 'social_vector_length', 'is_console_open', 'data_manipulation_risk', 'loan_limit_utilization', 'ip_region_name', 'ip_city', 'ip_owner', 'ip_domain', 'ip_mobile_brand', 'ip_usage_type', 'ip_net_speed', 'proxy', 'tor', 'ipv6_usage', 'ip_mismatch', 'dns_name', 'is_dns_local', 'ip_zip_code', 'virtual_machine', 'is_useragent_structure_issue', 'is_randomizer_issue', 'is_private_mode', 'botnet', 'connection_via_browser', 'connection_via_application', 'is_web_view_application_used', 'masquerading_as_desktop', 'device_vendor', 'mobile_device', 'touch_screen', 'screen_geometry_mismatch', 'is_regular_screen_resolution', 'nfc', 'pixel_language', 'os_name', 'browser_name', 'rendering_engine', 'region_app' ] 

In [70]:
for name in categorical :
    d = d.withColumn( name, f.when( f.col( name ).isNull(), 'Unknown' ).otherwise( f.col( name ) ) )

Missing values in numeric variables are replaced by 0.5 (as they have already been scaled to the interval [0, 1]). If the portion of missing values is significant (equal or greater than 5%), a new binary variable is produced (indicator of a missing value).

In [71]:
for name in numeric :
    n = d.select( name ).agg( f.sum( f.when( f.col( name ).isNull(), 1 ).otherwise( 0 ) ) ).collect()[ 0 ][ 0 ]
    if n / nrows >= 0.05 and n / nrows <= 0.95 :
        d = d.withColumn( name+'_null', f.when( f.col( name ).isNull(), 1 ).otherwise( 0 ) )
        d = d.withColumn( name, f.when( f.col( name ).isNull(), 0.5 ).otherwise( f.col( name ) ) )
    elif n > 0 :
        d = d.withColumn( name, f.when( f.col( name ).isNull(), 0.5 ).otherwise( f.col( name ) ) )

#### Categorical Variables with Large Number of Categories

In [72]:
many_categories = [ 'zip', 'ip_zip_code', 'region_app', 'ip_city', 'ip_owner', 'ip_region_name', 'ip_domain', 'dns_name', 'device_vendor' ]

In [73]:
share = d.filter( 'test = false' ).agg( f.mean( 'label' ) ).collect()[ 0 ][ 0 ]
print( share )

0.046399885255306944


In [74]:
for name in many_categories :
    nameenc = name+'_encoded'
    cat = d.filter( 'test = false' ).groupBy( name ).agg( f.sum( 'label' ).alias( 'positive' ), 
                                                          f.count( 'label' ).alias( 'total' ) )
    cat = cat.withColumn( nameenc, ( cat.positive + f.lit( 30 ) * f.lit( share ) ) / ( cat.total + f.lit( 30 ) ) )
    cat = cat.withColumnRenamed( name, 'new' )
    d = d.join( cat.select( 'new', nameenc ), f.col( name ) == cat.new, how='left' )
    d = d.drop( 'new' )
    d = d.withColumn( nameenc, f.when( f.col( nameenc ).isNull(), share ).otherwise( f.col( nameenc ) ) )

Dummy features are added in order to be used later in the variable selection process:

In [75]:
d = d.withColumn( 'DUMMY1', f.rand( seed=10000 ) ).\
      withColumn( 'DUMMY2', f.rand( seed=20000 ) ).\
      withColumn( 'DUMMY3', f.rand( seed=30000 ) )

In [76]:
d.coalesce(1).write.csv( '/home/demetrius/Documents/Python/Fraud Detection/Data/processed', sep=';', header=True )

In [3]:
d = sc.read.csv( '/home/demetrius/Documents/Python/Fraud Detection/Data/processed/part-00000-16f7e18f-f47e-4ff1-b42b-492aafe3d348-c000.csv',
                inferSchema=True, header=True, sep=';' )
numeric = [ 'amount_prep', 'term_prep', 'age', 'income', 'cl_education_type_id', 'antifraud_score', 'time', 'idx1_stop_markers', 'idx2_user_behaviour_markers', 'idx3_device_markers', 'idx4_connection_markers', 'idx5_device_quality', 'idx6_internet_infrastructure_quality', 'idx8_device_credentials_variability', 'total_applications_number', 'total_num_of_applications_with_browser_hash_in_1_day', 'total_num_of_applications_with_browser_hash_in_7_days', 'total_num_of_applications_with_browser_hash_in_30_days', 'logins_number', 'phones_number', 'devices_number', 'cities_number', 'zipcodes_number', 'cards_number', 'device_ageing_with_user', 'total_num_of_shortterm_credit_applications_in_1_day', 'total_num_of_shortterm_credit_applications_in_7_days', 'total_num_of_shortterm_credit_applications_in_30_days', 'total_num_of_shortterm_credit_applications_from_ip_in_1_day', 'total_num_of_shortterm_credit_applications_from_ip_in_7_days', 'total_num_of_shortterm_credit_applications_from_ip_in_30_days', 'total_num_of_banking_credit_applications_in_1_day', 'total_num_of_banking_credit_applications_in_7_days', 'total_num_of_banking_credit_applications_in_30_days', 'total_num_of_banking_credit_applications_from_ip_in_1_day', 'total_num_of_banking_credit_applications_from_ip_in_7_days', 'total_num_of_banking_credit_applications_from_ip_in_30_days', 'total_num_of_insurance_applications_from_ip_in_30_days', 'mail_login_length', 'time_on_page', 'number_of_hot_keys', 'number_of_corrections', 'browser_history_count', 'ram_productivity', 'storage_productivity', 'pure_avg_typing_speed', 'avg_typing_speed', 'single_click', 'double_click', 'context_menu', 'leave_page', 'observed_quarters', 'touch_dev', 'acc_dev', 'number_of_secondary_antifraud_markers', 'ip_longitude', 'ip_latitude', 'ip_ageing_in_months', 'is_canvas_blocker', 'display_width_via_pixel', 'display_height_via_pixel', 'display_width_from_manufacturer', 'display_height_from_manufacturer', 'usable_display_width', 'usable_display_height', 'diagonal_screen_size', 'display_ppi', 'display_pixel_ratio', 'color_depth_via_pixel', 'color_depth_from_manufacturer', 'ram_size', 'hardware_concurrency', 'camera_resolution', 'completeness', 'time_sine', 'time_cosine', 'device_age' ]
degenerated = [ 'total_num_of_shortterm_credit_applications_from_ip_in_1_day', 'total_num_of_banking_credit_applications_in_1_day', 'total_num_of_banking_credit_applications_in_7_days', 'total_num_of_banking_credit_applications_in_30_days', 'total_num_of_banking_credit_applications_from_ip_in_1_day', 'total_num_of_banking_credit_applications_from_ip_in_7_days', 'total_num_of_insurance_applications_from_ip_in_30_days', 'mail_login_length', 'number_of_hot_keys', 'number_of_corrections', 'total_num_of_shortterm_credit_applications_in_7_days', 'total_num_of_shortterm_credit_applications_in_1_day', 'cards_number', 'devices_number', 'phones_number', 'idx3_device_markers', 'idx1_stop_markers' ]
numeric = [ x for x in numeric if x not in degenerated ]
categorical = [ 'total_num_of_shortterm_credit_applications_from_ip_in_1_day', 'total_num_of_banking_credit_applications_in_1_day', 'total_num_of_banking_credit_applications_in_7_days', 'total_num_of_banking_credit_applications_in_30_days', 'total_num_of_banking_credit_applications_from_ip_in_1_day', 'total_num_of_banking_credit_applications_from_ip_in_7_days', 'total_num_of_insurance_applications_from_ip_in_30_days', 'mail_login_length', 'number_of_hot_keys', 'number_of_corrections', 'total_num_of_shortterm_credit_applications_in_7_days', 'total_num_of_shortterm_credit_applications_in_1_day', 'cards_number', 'devices_number', 'phones_number', 'idx3_device_markers', 'idx1_stop_markers', 'gender', 'cl_marital_status_id', 'cl_employment_type_id', 'cl_employment_area_id', 'zip', 'javascript_executable', 'ip_available', 'useragent_available', 'supporting_data_available', 'is_local_country', 'timezone_mismatch', 'ip_sim_regions_match', 'duplicating_device', 'duplicating_ip', 'duplicating_user', 'seen_before', 'same_phone', 'same_login', 'same_region', 'same_device', 'less_tenor_days', 'total_num_of_insurance_applications_in_1_day', 'total_num_of_insurance_applications_in_7_days', 'total_num_of_insurance_applications_in_30_days', 'total_num_of_insurance_applications_from_ip_in_1_day', 'total_num_of_insurance_applications_from_ip_in_7_days', 'foreign_language_used', 'frd', 'frd_from_diff_acc', 'npl90', 'npl90_from_diff_acc', 'is_ever_frd', 'is_mass_frd', 'global_ip_blacklist', 'global_isp_blacklist', 'user_defined_ip_blacklist', 'user_defined_deviceid_blacklist', 'user_defined_userid_blacklist', 'user_defined_isp_blacklist', 'user_defined_similar_deviceid_blacklist', 'suspicious_referral', 'suspicious_plugins', 'suspicious_fonts', 'cookie_clone', 'mail_login_marker', 'social_vector_length', 'is_console_open', 'data_manipulation_risk', 'loan_limit_utilization', 'ip_region_name', 'ip_city', 'ip_owner', 'ip_domain', 'ip_mobile_brand', 'ip_usage_type', 'ip_net_speed', 'proxy', 'tor', 'ipv6_usage', 'ip_mismatch', 'dns_name', 'is_dns_local', 'ip_zip_code', 'virtual_machine', 'is_useragent_structure_issue', 'is_randomizer_issue', 'is_private_mode', 'botnet', 'connection_via_browser', 'connection_via_application', 'is_web_view_application_used', 'masquerading_as_desktop', 'device_vendor', 'mobile_device', 'touch_screen', 'screen_geometry_mismatch', 'is_regular_screen_resolution', 'nfc', 'pixel_language', 'os_name', 'browser_name', 'rendering_engine', 'region_app' ] 
many_categories = [ 'zip', 'ip_zip_code', 'region_app', 'ip_city', 'ip_owner', 'ip_region_name', 'ip_domain', 'dns_name', 'device_vendor' ]
share = d.filter( 'test = false' ).agg( f.mean( 'label' ) ).collect()[ 0 ][ 0 ]

#### One-Hot Encoding

In [4]:
categorical = [ x for x in categorical if x not in many_categories ]

In [5]:
produced = [ 'income_null', 'idx8_device_credentials_variability_null', 'pure_avg_typing_speed_null', 'avg_typing_speed_null', 'single_click_null', 'leave_page_null', 'touch_dev_null', 'acc_dev_null', 'display_width_from_manufacturer_null', 'display_height_from_manufacturer_null', 'usable_display_width_null', 'usable_display_height_null', 'diagonal_screen_size_null', 'display_ppi_null', 'color_depth_from_manufacturer_null', 'ram_size_null', 'hardware_concurrency_null', 'camera_resolution_null', 'device_age_null', 'zip_encoded', 'ip_zip_code_encoded', 'region_app_encoded', 'ip_city_encoded', 'ip_owner_encoded', 'ip_region_name_encoded', 'ip_domain_encoded', 'dns_name_encoded', 'device_vendor_encoded', 'DUMMY1', 'DUMMY2', 'DUMMY3' ]

In [6]:
indexers = [ StringIndexer( stringOrderType='alphabetAsc', inputCol=name, 
                            outputCol=name+'_ind' ) for name in categorical ]

In [7]:
names_index = [ name + '_ind' for name in categorical ]
names_one_hot = [ name + '_oh' for name in categorical ]

In [8]:
encoder = OneHotEncoderEstimator( inputCols=names_index, outputCols=names_one_hot, dropLast=False )

In [9]:
vectorizer = VectorAssembler( inputCols=numeric+produced+names_one_hot, outputCol='features' )

In [10]:
pipe = Pipeline( stages=indexers+[ encoder, vectorizer ] )

In [11]:
oh = pipe.fit( d ).transform( d ).select( 'label', 'test', 'features' )

In [12]:
categorical_names = []
for name in categorical :
    for i in range( d.select( name ).distinct().count() ) :
        categorical_names.append( name+str( i ) )

In [13]:
variable_names = numeric+produced+categorical_names

#### Balancing Classes

In [14]:
balance = round( ( 1 - share ) / share )
print( balance )

21


In [15]:
oversampled_positive = oh.filter( 'label = 1' )
for i in range( 1, balance ) :
    oversampled_positive = oversampled_positive.union( oh.filter( 'label = 1' ) )
oversampled_positive.count()

16968

In [16]:
balanced = oh.filter( 'label = 0' ).union( oversampled_positive )
del oversampled_positive
balanced.select( 'label' ).summary( 'mean' ).show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|   mean|0.5055416517697533|
+-------+------------------+



#### Variable Selection

Summary dataframe for various trees:

In [17]:
trials = 40
t = pd.DataFrame({ 'leaf' : np.random.randint( 616, 2000, size=trials ),
                   'seed' : np.random.randint( 0, 4e9, size=trials ) })
t[ 'pr_auc' ] = 0
t[ 'nvars' ] = 0
t[ 'vars' ] = ''
t.head()

,leaf,seed,pr_auc,nvars,vars
0,1245,716207235,0,0,
1,1393,3714637870,0,0,
2,743,719291602,0,0,
3,1001,2629284994,0,0,
4,944,1642891591,0,0,


In [43]:
evaluator = BinaryClassificationEvaluator( metricName='areaUnderPR' )

In [ ]:
for i in t.index :
    print( i )
    tree = RandomForestClassifier( numTrees=1, maxDepth=10, maxBins=100, subsamplingRate=0.5, 
        featureSubsetStrategy='onethird', minInstancesPerNode=t.loc[ i, 'leaf' ], seed=t.loc[ i, 'seed' ] )
    model = tree.fit( balanced )
    t.loc[ i, 'pr_auc' ] = evaluator.evaluate( model.transform( oh ) )
    selected = [ variable_names[ j ] for j in model.featureImportances.indices ]
    t.loc[ i, 'nvars' ] = len( selected )
    t.loc[ i, 'vars' ] = ', '.join( selected )
    del( [ tree, model, selected ] )

1


In [ ]:
t.to_csv( '/home/dkulikov/Documents/Python/trees.csv' )

In [17]:
indexers = [ StringIndexer( stringOrderType='alphabetAsc', inputCol=name,
                            outputCol=name+'_ind' ) for name in [ 'calculator' ] ]

In [18]:
names_index = [ name + '_ind' for name in [ 'calculator' ] ]
names_one_hot = [ name + '_oh' for name in [ 'calculator' ] ]

In [19]:
encoder = OneHotEncoderEstimator( inputCols=names_index, outputCols=names_one_hot, dropLast=False )
vectorizer = VectorAssembler( inputCols=[ 'time_on_page', 'single_click_null', 'zip_encoded' ]+names_one_hot, 
                             outputCol='features' )
pipe = Pipeline( stages=indexers+[ encoder, vectorizer ] )
oh = pipe.fit( d ).transform( d ).select( 'id', 'amount', 'interest', 'label', 'test', 'features' )
oversampled_positive = oh.filter( 'label = 1' )
for i in range( 1, balance ) :
    oversampled_positive = oversampled_positive.union( oh.filter( 'label = 1' ) )
balanced = oh.filter( 'label = 0' ).union( oversampled_positive )
del oversampled_positive

#### Search of the Optimal Configuration of Neural Network 

In [97]:
trials = 2
t = pd.DataFrame({ 'maxiter' : np.random.choice( [ 100, 500, 1000, 2000, 3000 ], size=trials, replace=True ),
                   'stepsize' : np.random.choice( [ 0.01, 0.02, 0.03, 0.04, 0.05 ], size=trials, replace=True ),
                   'seed' : np.random.randint( 0, 4e9, size=trials ) })
t[ 'pr_auc' ] = 0
t.head()

,maxiter,stepsize,seed,pr_auc
0,1000,0.04,604230286,0
1,3000,0.01,1194179510,0


In [83]:
take_prob = f.udf( lambda v : float( v[ 1 ] ), FloatType() )

In [105]:
for i in t.index :
    print( i )
    nnet = MultilayerPerceptronClassifier( layers=[ , 4, 2, 2 ], blockSize=1000, solver='l-bfgs', 
               maxIter=t.loc[ i, 'maxiter' ], stepSize=t.loc[ i, 'stepsize' ],  seed=t.loc[ i, 'seed' ] )
    model = nnet.fit( balanced.filter( 'test = false' ) )
    predicted = model.transform( oh.filter( 'test = true' ) )
    if i == 0 :
        predictions = predicted.withColumn( 'prob_1', take_prob( 'probability' ) ).\
                                withColumnRenamed( 'prob_1', 'i0' ).select( 'id', 'i0' ) 
    else :
        predictions = predicted.withColumn( 'prob_1', take_prob( 'probability' ) ).\
                                withColumnRenamed( 'prob_1', 'i'+str( i ) ).select( 'id', 'i'+str( i ) ).\
                                join( predictions, on='id', how='inner' )
    t.loc[ i, 'pr_auc' ] = evaluator.evaluate( predicted )
    del( [ nnet, model, predicted ] )

1


#### Evaluation of Impact of the Model

In [110]:
e = predictions.select( 'id', 'i2' ).join( oh, on='id', how='inner' ).withColumnRenamed( 'i2', 'prob' )

| Stretch/Untouched | ProbDistribution | Accuracy |
| :- | :-: | :-: |
| <b>Stretched | Gaussian | .843 |

In [111]:
a.show()

+-------+----------+------+------------------+-----+----+--------------------+
|     id|      prob|amount|          interest|label|test|            features|
+-------+----------+------+------------------+-----+----+--------------------+
|8597257| 0.5905551| 170.0|              51.0|    0|true|[0.25621030977513...|
|8735162|0.27867174| 360.0|              54.0|    0|true|[0.25621030977513...|
|8661524|0.41102436|  50.0|              15.0|    0|true|(7,[0,2,5],[0.482...|
|8784764| 0.5127556|  40.0|              12.0|    0|true|(7,[0,2,5],[0.604...|
|8532156|0.36814708| 120.0|              18.0|    0|true|(7,[0,2,5],[0.488...|
|8670872|0.48777193| 360.0|             108.0|    0|true|(7,[0,2,4],[0.604...|
|8816556| 0.7479879| 120.0|              26.4|    0|true|(7,[0,2,6],[0.361...|
|8661153|0.35349476| 130.0|              28.6|    0|true|(7,[0,2,5],[0.482...|
|8533925| 0.3696772|  20.0|               5.4|    0|true|(7,[0,2,3],[0.604...|
|8217899| 0.1859771|  60.0|              18.0|    1|

In [108]:
predictions.show()

+-------+----------+----------+----------+
|     id|        i2|        i1|        i0|
+-------+----------+----------+----------+
|8597257| 0.5905551| 0.5719692| 0.5719691|
|8735162|0.27867174| 0.3086225|0.30862233|
|8661524|0.41102436|0.42394298|0.42394271|
|8784764| 0.5127556| 0.5163836|0.51638293|
|8532156|0.36814708|0.38830087|0.38830066|
|8670872|0.48777193| 0.4983167|0.49831694|
|8816556| 0.7479879|0.70302516| 0.7030256|
|8661153|0.35349476| 0.3754968|0.37549666|
|8533925| 0.3696772|0.39808446|0.39808476|
|8217899| 0.1859771|0.24508938|0.24508962|
|8495506|0.64204884| 0.6110017|  0.611002|
|8536849|0.20325872| 0.2536071|0.25360695|
|8732382|0.46209446|0.45246074|0.45246074|
|8763272| 0.6600779|0.63215256|0.63215244|
|8757303|0.28396577|0.33786002|0.33786055|
|8811079| 0.3586166|0.37984592|0.37984577|
|8747002|0.45117956|0.46545094|0.46545163|
|8715184| 0.3627789|0.38222328|0.38222358|
|8741869| 0.3131061|0.34927565|0.34927607|
|8432824|0.97074986| 0.9444955|0.94449544|
+-------+--

In [103]:
model.weights
# 'time_on_page', 'single_click_null', 'zip_encoded', 'change', 'default', 'exact', 'right'

DenseVector([1.3181, -1.1299, 0.6661, -0.2485, -5.9002, 6.4373, 0.0332, -0.1943, 0.2323, 0.5482, 0.3189, -0.1023, 0.1903, 0.5176, -0.6478, 0.0681])

In [107]:
model.weights

DenseVector([1.7287, -1.5405, 0.7902, -0.3727, -7.1786, 7.7157, 0.0268, -0.1879, 0.1728, 0.6077, 0.3294, -0.1128, 0.1237, 0.5843, -0.7698, 0.1901])

In [63]:
s = sc.createDataFrame( [ ( 1, 'default', 200, 0 ),
                          ( 2, 'exact', 200, 0 ),
                          ( 3, 'right', 300, 1 ),
                          ( 4, 'default', 100, 0 ) ],
                          [ 'id', 'calculator', 'amount', 'income_null' ] )
s.show()

+---+----------+------+-----------+
| id|calculator|amount|income_null|
+---+----------+------+-----------+
|  1|   default|   200|          0|
|  2|     exact|   200|          0|
|  3|     right|   300|          1|
|  4|   default|   100|          0|
+---+----------+------+-----------+



In [18]:
d.filter( 'amount = 230' ).select( 'DUMMY1', 'DUMMY2', 'DUMMY3' ).show()

+-------------------+--------------------+--------------------+
|             DUMMY1|              DUMMY2|              DUMMY3|
+-------------------+--------------------+--------------------+
| 0.9101417418112615|  0.8702175563203732|  0.5976111546866278|
| 0.5133452710932718|  0.9236611454788727|  0.9880197032482291|
| 0.5684106677034393| 0.08735706117007125|   0.610442828365377|
| 0.6373286000654003|  0.9992155172779373| 0.28154566586246166|
| 0.4549122148212914|  0.5502734341874344| 0.42600482777192306|
| 0.9357786322462535|  0.5528826321355369|  0.2614839594381747|
| 0.9385201811416523|0.008991653987770909|  0.6519034659964507|
| 0.7582740076462351| 0.17789923672516306| 0.23379678889855182|
|0.26089303563584465| 0.10963337991619893|  0.7054024879087801|
|  0.502200813872946| 0.12492122419067886|  0.9001188838061123|
| 0.6766159758031817|  0.7675256905637611|  0.4028668954032355|
| 0.5993694543340292|  0.9471668555786584|   0.987575207855827|
| 0.8989815988679949|  0.943658832683035